# Import

In [16]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler
from scipy.stats.mstats import winsorize
from IPython.display import display
from pathlib import Path

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


# Load Data

In [2]:
data_dir = Path('../../data/raw/fa')
if not data_dir.exists():
    print('Data directory not found:', data_dir.resolve())
else:
    csv_files = sorted(data_dir.glob('*.csv'))
    print(f'Found {len(csv_files)} CSV file(s)')
    dfs = {}
    for f in csv_files:
        try:
            df = pd.read_csv(f)
            dfs[f.stem] = df
            print(f'Loaded {f.name}')
        except Exception as e:
            print(f'Failed to load {f.name}:', e)


Found 3 CSV file(s)
Loaded bangcandoiketoan.csv
Loaded bangluuchuyentiente.csv
Loaded baocaotaichinh.csv


# I. Data Understanding (Tìm hiểu Dữ liệu)

Mục tiêu: Làm quen với dữ liệu, xác định các vấn đề về chất lượng và khám phá những insight đầu tiên.

## 1.1 Describe Data (Mô tả Dữ liệu)

Thông tin khái quát về dữ liệu:
* Bao nhiêu hàng, bao nhiêu cột?
* Tên các cột là gì?
* Kiểu dữ liệu của mỗi cột là gì?

In [3]:
print("Cấu trúc tổng quan của dữ liệu:")
for name, df in dfs.items():
	print(f'\nDataFrame: {name}')
	print(f'Shape: {df.shape}')
	df.info()
	display(df.head())


Cấu trúc tổng quan của dữ liệu:

DataFrame: bangcandoiketoan
Shape: (12431, 76)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12431 entries, 0 to 12430
Data columns (total 76 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   CP                                                           12431 non-null  object 
 1   Năm                                                          12431 non-null  int64  
 2   Kỳ                                                           12431 non-null  int64  
 3   TÀI SẢN NGẮN HẠN (đồng)                                      12112 non-null  float64
 4   Tiền và tương đương tiền (đồng)                              12431 non-null  int64  
 5   Giá trị thuần đầu tư ngắn hạn (đồng)                         11283 non-null  float64
 6   Các khoản phải thu ngắn hạn (đồng)                           12112 non-null  float64
 

,CP,Năm,Kỳ,TÀI SẢN NGẮN HẠN (đồng),Tiền và tương đương tiền (đồng),Giá trị thuần đầu tư ngắn hạn (đồng),Các khoản phải thu ngắn hạn (đồng),Hàng tồn kho ròng,Tài sản lưu động khác,TÀI SẢN DÀI HẠN (đồng),...,Vốn của tổ chức tín dụng,Quỹ của tổ chức tín dụng,Chênh lệch tỷ giá hối đoái,Lợi thế thương mại,Vốn Ngân sách nhà nước và quỹ khác,_Các quỹ khác,Cổ đông thiểu số,Đầu tư vào công ty con,Tài sản cố định thuê tài chính,Chênh lệch đánh giá lại tài sản
0,AAA,2025,3,4.377037e+12,1741373916584,1.864000e+11,1.531252e+12,7.572832e+11,1.607281e+11,7.864701e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAA,2025,2,4.553020e+12,1706426051965,2.294076e+11,1.475455e+12,9.872324e+11,1.544995e+11,7.606508e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAA,2025,1,4.793960e+12,2106077190958,1.661907e+11,1.551511e+12,8.275360e+11,1.426454e+11,7.390793e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAA,2024,4,6.426369e+12,2419517905105,7.186394e+11,1.801012e+12,1.286443e+12,2.007563e+11,7.341846e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAA,2024,3,6.956715e+12,1945532019980,1.346697e+12,2.303219e+12,1.138806e+12,2.224600e+11,6.075525e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



DataFrame: bangluuchuyentiente
Shape: (12327, 47)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12327 entries, 0 to 12326
Data columns (total 47 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   Mã CK                                                                12327 non-null  object 
 1   CP                                                                   12327 non-null  object 
 2   Năm                                                                  12327 non-null  int64  
 3   Kỳ                                                                   12327 non-null  int64  
 4   Lãi/Lỗ ròng trước thuế                                               9659 non-null   float64
 5   Khấu hao TSCĐ                                                        10924 non-null  float64
 6   Dự phòng RR tín dụng                               

,Mã CK,CP,Năm,Kỳ,Lãi/Lỗ ròng trước thuế,Khấu hao TSCĐ,Dự phòng RR tín dụng,Lãi/Lỗ chênh lệch tỷ giá chưa thực hiện,Lãi/Lỗ từ thanh lý tài sản cố định,Lãi/Lỗ từ hoạt động đầu tư,...,Ảnh hưởng của chênh lệch tỷ giá,Tiền và tương đương tiền cuối kỳ,Thu lãi và cổ tức,"Chi trả cho việc mua lại, trả cổ phiếu",_Tăng/Giảm các khoản phải thu,_Tăng/Giảm các khoản phải trả,(Lãi)/lỗ các hoạt động khác,Lưu chuyển tiền thuần từ HĐKD trước thuế,Chi từ các quỹ của TCTD,Cố tức đã nhận
0,AAA,AAA,2025,3,1.561775e+11,1.234796e+11,0.000000e+00,2.198127e+10,0.0,-3.205606e+10,...,-1.996808e+10,1.741374e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAA,AAA,2025,2,2.110003e+11,1.241982e+11,-5.335130e+07,-2.893948e+10,0.0,-7.310252e+10,...,2.912336e+10,1.706426e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAA,AAA,2025,1,6.849024e+10,1.316839e+11,-1.012389e+09,-8.222065e+08,0.0,8.378028e+10,...,-1.415966e+08,2.106077e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAA,AAA,2024,4,9.831536e+10,1.408386e+11,2.503038e+09,-8.992149e+09,0.0,6.956733e+10,...,1.494763e+10,2.419518e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAA,AAA,2024,3,-2.791523e+10,8.180615e+10,1.433809e+08,2.164394e+10,0.0,-2.879037e+10,...,-3.929451e+10,1.945532e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



DataFrame: baocaotaichinh
Shape: (12019, 47)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12019 entries, 0 to 12018
Data columns (total 47 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   CP                                                12019 non-null  object 
 1   Năm                                               12019 non-null  int64  
 2   Kỳ                                                12019 non-null  int64  
 3   Tăng trưởng doanh thu (%)                         11990 non-null  float64
 4   Doanh thu (đồng)                                  12013 non-null  float64
 5   Lợi nhuận sau thuế của Cổ đông công ty mẹ (đồng)  12019 non-null  int64  
 6   Tăng trưởng lợi nhuận (%)                         11996 non-null  float64
 7   Thu nhập tài chính                                11722 non-null  float64
 8   Chi phí tiền lãi vay                              

,CP,Năm,Kỳ,Tăng trưởng doanh thu (%),Doanh thu (đồng),Lợi nhuận sau thuế của Cổ đông công ty mẹ (đồng),Tăng trưởng lợi nhuận (%),Thu nhập tài chính,Chi phí tiền lãi vay,Doanh thu bán hàng và cung cấp dịch vụ,...,Chứng khoán đầu tư,Hoạt động khác,Chi phí hoạt động khác,Lãi/lỗ thuần từ hoạt động khác,Cố tức đã nhận,Tổng thu nhập hoạt động,LN từ HĐKD trước CF dự phòng,Chi phí dự phòng rủi ro tín dụng,Thuế TNDN,Lãi cơ bản trên cổ phiếu
0,AAA,2025,3,-0.257602,2.371755e+12,117554438469,4.392505,2.705027e+10,-2.423012e+10,2.371755e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAA,2025,2,-0.169946,2.310908e+12,131689030063,-0.030899,7.333893e+10,-3.663940e+10,2.310908e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAA,2025,1,0.301104,3.857615e+12,45239813399,-0.664160,3.764767e+10,-3.786021e+10,3.857615e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAA,2024,4,0.430535,3.844069e+12,76186464918,-0.187543,9.124983e+10,-4.978958e+10,3.844069e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAA,2024,3,-0.095326,3.194721e+12,21799597923,-0.779913,3.351684e+10,-3.390523e+10,3.194721e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dưới đây là tổng hợp phân tích về cấu trúc và đặc điểm của cả ba bộ dữ liệu bạn đã cung cấp: `bangcandoiketoan` (Bảng Cân đối Kế toán), `bangluuchuyentiente` (Báo cáo Lưu chuyển Tiền tệ), và `baocaotaichinh` (Báo cáo Kết quả Hoạt động Kinh doanh/Tổng hợp).

***

### 1. Tóm tắt Cấu trúc Chung

| DataFrame | Số lượng Bản ghi | Số lượng Cột | Mục đích chính | Độ Hoàn chỉnh Tổng thể |
| :--- | :--- | :--- | :--- | :--- |
| **Bảng Cân đối Kế toán** | 12.431 | 76 | Tài sản, Nợ phải trả, Vốn Chủ sở hữu tại một thời điểm. | Rất tốt cho các chỉ tiêu tổng hợp. |
| **Lưu chuyển Tiền tệ** | 12.327 | 47 | Dòng tiền ròng từ 3 hoạt động (Kinh doanh, Đầu tư, Tài chính) trong một kỳ. | Tốt cho các chỉ tiêu dòng tiền chính. |
| **Báo cáo Tài chính (KQKD)** | 12.019 | 47 | Kết quả hoạt động (Doanh thu, Chi phí, Lợi nhuận) trong một kỳ. | Tốt cho các chỉ tiêu lợi nhuận chính. |

**Nhận xét:** Cả ba bộ dữ liệu đều có **quy mô lớn** (hơn 12.000 bản ghi) và **giàu thuộc tính** (từ 47 đến 76 cột), cung cấp cái nhìn toàn diện về tài chính của các công ty trong nhiều kỳ.

***

### 2. Đặc điểm Quan trọng và Vấn đề Tiềm ẩn

Điểm chung lớn nhất và cần lưu ý nhất trong cả ba bộ dữ liệu là sự **kết hợp dữ liệu từ các loại hình doanh nghiệp khác nhau**, dẫn đến sự phân bố không đồng đều về độ hoàn chỉnh của các cột:

#### a) Độ Hoàn chỉnh Tuyệt vời (Dữ liệu Core)

Các chỉ tiêu **tổng hợp và cơ bản** của BCTC đều rất đầy đủ (Non-Null Count $\approx$ 12.000 - 12.431):
* **BCĐKT:** Tổng Tài sản, Nợ phải trả, Vốn chủ sở hữu.
* **LCTT:** Dòng tiền thuần từ HĐKD/Đầu tư/Tài chính, Tiền và tương đương tiền cuối kỳ.
* **BCKQKD:** Doanh thu thuần, Lãi gộp, Chi phí quản lý DN, Lợi nhuận trước/sau thuế.

Đây là nền tảng vững chắc để thực hiện các phân tích về quy mô, cấu trúc vốn, và khả năng sinh lời ở cấp độ cao.

#### b) Độ Hoàn chỉnh Thấp (Các chỉ tiêu Đặc thù)

Một số lượng lớn cột có số lượng Non-Null rất thấp, thường chỉ **dưới 500** bản ghi, cho thấy chúng chỉ áp dụng cho một tập hợp rất nhỏ các công ty:

* **Các chỉ tiêu Tài chính/Ngân hàng:** Xuất hiện trong cả ba bộ data (ví dụ: Chứng khoán kinh doanh, Tiền gửi tại NHNN Việt Nam, Thu nhập lãi thuần, Chi phí dự phòng rủi ro tín dụng, Chi từ các quỹ của TCTD).
    * **Ý nghĩa:** Tập dữ liệu này bao gồm cả **Tổ chức Tín dụng (TCTD)**. Các cột này là **dữ liệu cốt lõi** nếu bạn phân tích Ngân hàng nhưng gần như vô dụng (NaN) cho các công ty sản xuất/thương mại.
* **Các chỉ tiêu Kế toán Chi tiết/Ít phổ biến:** (Ví dụ: Lãi/Lỗ từ thanh lý tài sản cố định, Lợi thế thương mại, Đầu tư vào công ty con).
    * **Ý nghĩa:** Những hoạt động này không xảy ra thường xuyên hoặc không được báo cáo chi tiết theo chuẩn mực kế toán cho mọi công ty.

***

### 3. Đề xuất Xử lý Dữ liệu và Phân tích Tiếp theo

Để khai thác hiệu quả 3 bộ dữ liệu này, cần thực hiện các bước sau:

#### a) Phân tách (Segmentation) Dữ liệu
* **Bước quan trọng nhất:** Dựa vào các cột đặc thù (ví dụ: `Thu nhập lãi và các khoản tương tự` trong `baocaotaichinh`), **xác định và phân tách** nhóm công ty TCTD/Ngân hàng ra khỏi nhóm công ty thông thường.
* **Lợi ích:** Phân tích từng nhóm riêng biệt sẽ loại bỏ được phần lớn dữ liệu khuyết thiếu (NaN) không liên quan.

#### b) Xử lý Giá trị Khuyết thiếu (NaN)
* **Đối với các cột tài chính:** Nếu một công ty không có giá trị cho một chỉ tiêu (ví dụ: **Các khoản giảm trừ doanh thu**), điều đó thường có nghĩa là giá trị đó bằng **0**. Nên sử dụng phương pháp **điền giá trị 0** cho các cột tài chính có độ hoàn chỉnh ở mức trung bình (trên 7.000 non-null) hoặc sau khi đã phân tách nhóm dữ liệu.

#### c) Chuẩn bị cho Tính toán Chỉ số
* Sử dụng **Mã CP**, **Năm**, **Kỳ** làm khóa (key) để **kết hợp** ba DataFrame lại với nhau.
* Mỗi chỉ số tài chính (Ví dụ: Tỷ lệ thanh toán hiện hành, ROE, Tỷ suất lợi nhuận gộp) sẽ yêu cầu dữ liệu từ ít nhất hai báo cáo. 


## 1.2 Data Preprocessing (Tiền xử lý Dữ liệu)

In [10]:
# BƯỚC 1: TIÊU CHUẨN HÓA TÊN CỘT KHÓA

if 'Mã CK' in dfs['bangluuchuyentiente'].columns:
    dfs['bangluuchuyentiente'].drop(columns=['CP'], inplace=True)
    dfs['bangluuchuyentiente'].rename(columns={'Mã CK': 'CP'}, inplace=True)

# BƯỚC 2: XỬ LÝ DỮ LIỆU KHUYẾT THIẾU (IMPUTATION)

for name, df in dfs.items():
    float_cols = df.select_dtypes(include=['float64']).columns
    df[float_cols] = df[float_cols].fillna(0)

print("\nTổng số lượng NaN còn lại sau khi xử lý:")
for name, df in dfs.items():
    print(f"- DataFrame **{name}**: {df.isnull().sum().sum()}")
    
# BƯỚC 3: GIA NHẬP (JOINING) DỮ LIỆU

df_combined = dfs['bangcandoiketoan'].copy()

df_combined = df_combined.merge(
    dfs['bangluuchuyentiente'], 
    on=['CP', 'Năm', 'Kỳ'], 
    how='left', 
    suffixes=('_BCDKT', '_LCTT')
)

df_combined = df_combined.merge(
    dfs['baocaotaichinh'], 
    on=['CP', 'Năm', 'Kỳ'], 
    how='left', 
    suffixes=('_LCTT', '_BCTC')
)

print("\nCấu trúc của DataFrame Kết hợp:")
df_combined.shape
df_combined.info()
display(df_combined.head())



Tổng số lượng NaN còn lại sau khi xử lý:
- DataFrame **bangcandoiketoan**: 0
- DataFrame **bangluuchuyentiente**: 0
- DataFrame **baocaotaichinh**: 0

Cấu trúc của DataFrame Kết hợp:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12719 entries, 0 to 12718
Columns: 163 entries, CP to Lãi cơ bản trên cổ phiếu
dtypes: float64(152), int64(10), object(1)
memory usage: 15.8+ MB


,CP,Năm,Kỳ,TÀI SẢN NGẮN HẠN (đồng),Tiền và tương đương tiền (đồng),Giá trị thuần đầu tư ngắn hạn (đồng),Các khoản phải thu ngắn hạn (đồng),Hàng tồn kho ròng,Tài sản lưu động khác,TÀI SẢN DÀI HẠN (đồng),...,Chứng khoán đầu tư_BCTC,Hoạt động khác,Chi phí hoạt động khác,Lãi/lỗ thuần từ hoạt động khác,Cố tức đã nhận_BCTC,Tổng thu nhập hoạt động,LN từ HĐKD trước CF dự phòng,Chi phí dự phòng rủi ro tín dụng,Thuế TNDN,Lãi cơ bản trên cổ phiếu
0,AAA,2025,3,4.377037e+12,1741373916584,1.864000e+11,1.531252e+12,7.572832e+11,1.607281e+11,7.864701e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AAA,2025,3,4.377037e+12,1741373916584,1.864000e+11,1.531252e+12,7.572832e+11,1.607281e+11,7.864701e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AAA,2025,3,4.377037e+12,1741373916584,1.864000e+11,1.531252e+12,7.572832e+11,1.607281e+11,7.864701e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AAA,2025,2,4.553020e+12,1706426051965,2.294076e+11,1.475455e+12,9.872324e+11,1.544995e+11,7.606508e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AAA,2025,2,4.553020e+12,1706426051965,2.294076e+11,1.475455e+12,9.872324e+11,1.544995e+11,7.606508e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# BƯỚC 4: XỬ LÝ TRÙNG LẶP (DUPLICATES)
duplicate_count = df_combined.duplicated(subset=['CP', 'Năm', 'Kỳ']).sum()
if duplicate_count > 0:
    df_combined.drop_duplicates(subset=['CP', 'Năm', 'Kỳ'], keep='first', inplace=True)

# BƯỚC 5: PHÂN TÍCH PHÂN NHÓM (SEGMENTATION)
financial_cols = [
    'Thu nhập lãi thuần',  
    'Thu nhập từ hoạt động dịch vụ', 
    'Chi phí dự phòng rủi ro tín dụng',
    'Tiền gửi tại ngân hàng nhà nước Việt Nam',
    'Tiền gửi tại các TCTD khác và cho vay các TCTD khác',
    'Chi từ các quỹ của TCTD',
]
df_combined['is_financial'] = (df_combined[financial_cols].abs().sum(axis=1) > 0).astype(int)

# BƯỚC 6: TẠO DATASETS RIÊNG BIỆT CHO PHÂN TÍCH
df_financial = df_combined[df_combined['is_financial'] == 1].copy()
df_non_financial = df_combined[df_combined['is_financial'] == 0].copy()

# BƯỚC 7: IN KÍCH THƯỚC DATASETS ĐÃ PHÂN NHÓM
print(f"Kích thước DataFrame sau khi xử lý trùng lặp: {df_combined.shape}")
print(f"Kích thước DataFrame Tài chính: {df_financial.shape}")
print(f"Kích thước DataFrame Phi Tài chính: {df_non_financial.shape}")


Kích thước DataFrame sau khi xử lý trùng lặp: (12431, 164)
Kích thước DataFrame Tài chính: (319, 164)
Kích thước DataFrame Phi Tài chính: (12112, 164)


Dữ liệu đã được **xử lý và phân nhóm thành công**!

Kết quả này xác nhận những nhận định ban đầu về cấu trúc dữ liệu và đã chuẩn bị nền tảng rất tốt cho các bước phân tích tiếp theo.

### 1. Tổng quan về Dữ liệu đã Xử lý

| Đặc điểm | Kết quả | Nhận xét |
| :--- | :--- | :--- |
| **Kích thước Ban đầu** | 12.719 bản ghi | (Kết quả sau khi join bị trùng lặp) |
| **Kích thước sau Trùng lặp** | **(12431, 164)** | Số lượng bản ghi đã được chuẩn hóa về mức hợp lý, cho thấy dữ liệu ban đầu có khoảng **12.431** bản ghi duy nhất. |
| **Số lượng Đặc trưng (Cột)** | **164** | (76 BCĐKT + 47 LCTT + 47 KQKD) - 3 cột key - 1 cột loại bỏ + 1 cột phân nhóm. Tổng cộng 164 chỉ tiêu tài chính sẵn sàng để phân tích. |

---

### 2. Phân tích Phân nhóm (Segmentation)

Việc phân nhóm cho thấy sự tách biệt rõ ràng giữa hai loại hình doanh nghiệp:

| Nhóm Doanh nghiệp | Kích thước DataFrame | Tỷ lệ trong tổng số | Nhận định Phân tích |
| :--- | :--- | :--- | :--- |
| **Tài chính/Ngân hàng** | **319** bản ghi | 2.57\%$ | Nhóm này chứa các **Tổ chức Tín dụng (TCTD)**. Phân tích cần tập trung vào các chỉ số **CAMELS** (Capital Adequacy, Asset Quality, Management, Earnings, Liquidity, Sensitivity) và **lợi nhuận lãi thuần (NIM)**. |
| **Phi Tài chính** | **12112** bản ghi | $97.43\%$ | Đây là nhóm đa số (sản xuất, thương mại, dịch vụ). Phân tích cần tập trung vào các chỉ số truyền thống như **ROA, ROE, Tỷ lệ Thanh khoản, Tỷ lệ Đòn bẩy** và **Tăng trưởng Doanh thu/Lợi nhuận**. |

**$ \rightarrow $ Kết luận:** Việc phân nhóm thành công với **319** bản ghi tài chính đã xác nhận rằng các chỉ tiêu có Non-Null Count thấp (khoảng $\approx 300$ - $350$ trong phân tích trước) chính xác là dữ liệu của TCTD.

---

### 3. Bước Tiếp theo: Tính toán Chỉ số Tài chính (Feature Engineering)

### Đề xuất tính toán Chỉ số cho Nhóm Phi Tài chính (`df_non_financial`):

| Nhóm Chỉ số | Tên Chỉ số (Ví dụ) | Công thức (Ví dụ) |
| :--- | :--- | :--- |
| **Hiệu suất** | ROA | Lợi nhuận thuần / TỔNG CỘNG TÀI SẢN |
| | ROE | Lợi nhuận thuần / VỐN CHỦ SỞ HỮU |
| **Thanh khoản** | Tỷ lệ Thanh toán Hiện hành | TÀI SẢN NGẮN HẠN / Nợ ngắn hạn |
| **Đòn bẩy** | Tỷ lệ Nợ/Vốn Chủ sở hữu (D/E) | NỢ PHẢI TRẢ / VỐN CHỦ SỞ HỮU |
| **Hoạt động** | Vòng quay Hàng tồn kho | Giá vốn hàng bán / Hàng tồn kho ròng |

In [11]:
# BƯỚC 6: TÍNH TOÁN CHỈ SỐ TÀI CHÍNH CƠ BẢN (CHO NHÓM PHI TÀI CHÍNH)

# 1. Nhóm Chỉ số Hiệu suất (Profitability Ratios)
df_non_financial['ROA'] = df_non_financial['Lợi nhuận thuần'] / df_non_financial['TỔNG CỘNG TÀI SẢN (đồng)']
df_non_financial['ROE'] = df_non_financial['Lợi nhuận thuần'] / df_non_financial['VỐN CHỦ SỞ HỮU (đồng)']

# 2. Nhóm Chỉ số Thanh khoản (Liquidity Ratios)
df_non_financial['Current_Ratio'] = df_non_financial['TÀI SẢN NGẮN HẠN (đồng)'] / df_non_financial['Nợ ngắn hạn (đồng)']

# 3. Nhóm Chỉ số Đòn bẩy (Leverage Ratios)
epsilon = 1e-9 
df_non_financial['D_E_Ratio'] = df_non_financial['NỢ PHẢI TRẢ (đồng)'] / df_non_financial['VỐN CHỦ SỞ HỮU (đồng)'].replace(0, epsilon)

# 4. Nhóm Chỉ số Hoạt động (Activity Ratios)
df_non_financial['Inventory_Turnover'] = df_non_financial['Giá vốn hàng bán'] / df_non_financial['Hàng tồn kho ròng'].replace(0, epsilon)

print("Đã thêm 5 cột chỉ số mới vào DataFrame Phi Tài chính.")
df_non_financial.shape
display(df_non_financial[['CP', 'Năm', 'Kỳ', 'ROA', 'ROE', 'Current_Ratio', 'D_E_Ratio', 'Inventory_Turnover']].head())


Đã thêm 5 cột chỉ số mới vào DataFrame Phi Tài chính.


,CP,Năm,Kỳ,ROA,ROE,Current_Ratio,D_E_Ratio,Inventory_Turnover
0,AAA,2025,3,0.010246,0.020842,1.407069,1.034227,-2.669894
3,AAA,2025,2,0.014882,0.030961,1.427082,1.080434,-1.981129
6,AAA,2025,1,0.004556,0.009195,1.568238,1.018252,-4.099753
9,AAA,2024,4,0.004625,0.010210,1.555045,1.207763,-2.638215
12,AAA,2024,3,-0.001972,-0.004230,1.681821,1.144685,-2.500920


In [15]:
# BƯỚC 7: XỬ LÝ NGOẠI LAI (WINSORIZATION)

financial_ratios = ['ROA', 'ROE', 'Current_Ratio', 'D_E_Ratio', 'Inventory_Turnover']
df_non_financial_cleaned = df_non_financial.copy()

# Áp dụng Winsorization cho từng cột chỉ số
for col in financial_ratios:
    # Winsorize ở mức 1% (cắt 1% giá trị thấp nhất và 1% giá trị cao nhất)
    # Giữ lại giá trị NaN nếu có, mặc dù chúng ta đã điền 0 ở bước trước
    df_non_financial_cleaned[col] = winsorize(
        df_non_financial_cleaned[col], 
        limits=[0.01, 0.01]
    )

print("Thống kê mô tả sau khi xử lý ngoại lai:")
display(df_non_financial_cleaned[financial_ratios].describe().T)

# BƯỚC 8: CHUẨN HÓA DỮ LIỆU (STANDARD SCALER)

scaler = StandardScaler()

df_non_financial_cleaned[financial_ratios] = scaler.fit_transform(
    df_non_financial_cleaned[financial_ratios]
)

print(f"Đã xử lý ngoại lai và chuẩn hóa {len(financial_ratios)} chỉ số.")
display(df_non_financial_cleaned[financial_ratios].head())


Thống kê mô tả sau khi xử lý ngoại lai:


,count,mean,std,min,25%,50%,75%,max
ROA,11678.0,1.043916e-02,2.730659e-02,-6.893640e-02,0.000515,0.006739,0.018092,1.127050e+00
ROE,11678.0,2.554938e-02,3.989782e-01,-2.441084e-01,0.001781,0.015456,0.035726,4.167813e+01
Current_Ratio,12112.0,3.244879e+00,5.781528e+00,1.743036e-01,1.138818,1.570122,2.760303,4.228952e+01
D_E_Ratio,12112.0,1.349055e+00,2.295495e+00,-5.962951e+00,0.325533,0.814613,1.718649,1.429201e+01
Inventory_Turnover,11678.0,-3.791437e+18,2.250643e+19,-1.796742e+20,-4.988271,-1.343243,-0.478810,2.752997e+20


Đã xử lý ngoại lai và chuẩn hóa 5 chỉ số.


,ROA,ROE,Current_Ratio,D_E_Ratio,Inventory_Turnover
0,-0.007080,-0.011798,-0.317889,-0.137156,0.168467
3,0.162716,0.013565,-0.314428,-0.117026,0.168467
6,-0.215462,-0.040993,-0.290012,-0.144115,0.168467
9,-0.212937,-0.038447,-0.292294,-0.061554,0.168467
12,-0.454538,-0.074642,-0.270365,-0.089035,0.168467


## 1.3 Lưu trữ Dữ liệu đã Tiền xử lý

In [18]:
OUTPUT_PATH = "../../data/processed/fa/"

os.makedirs(OUTPUT_PATH, exist_ok=True)
# 1. Lưu DataFrame Kết hợp đã xử lý
df_combined.to_csv(
    os.path.join(OUTPUT_PATH, "combined_data.csv"),
    index=False
)
print(f"Đã lưu combined_data.csv")

# 2. Lưu DataFrame Phi Tài chính đã xử lý (đã chuẩn hóa)
df_non_financial_cleaned.to_csv(
    os.path.join(OUTPUT_PATH, "non_financial_data.csv"),
    index=False
)
print(f"Đã lưu non_financial_data.csv")

# 3. Lưu DataFrame Tài chính (chỉ phân nhóm, chưa scale do tính toán khác)
df_financial.to_csv(
    os.path.join(OUTPUT_PATH, "financial_data.csv"),
    index=False
)
print(f"Đã lưu financial_data.csv")


Đã lưu combined_data.csv
Đã lưu non_financial_data.csv
Đã lưu financial_data.csv


# II. Exploratory Data Analysis (Phân tích Khám phá Dữ liệu)